In [ ]:
import pandas as pd
import numpy as np
import json

# Load the training data
train_path = '/home/data/train.json'

# Read the JSON file - it's a JSON array, not line-delimited
with open(train_path, 'r') as f:
    train_data = json.load(f)

# Convert to DataFrame
df = pd.DataFrame(train_data)
print(f"Dataset shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")